In [1]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [3]:
print ('import statent grid')
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)

print('read statpop that has been pre-joined with statent')
statpop = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
)

import statent grid
read statpop that has been pre-joined with statent


In [3]:
print('read transport network default')
transport_network_default = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip')
    #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip')
)

read transport network default


In [3]:
print('read transport network cycling')
transport_network_cycling = r5py.TransportNetwork(
    os.path.join(outputs_path, 'before_oneway_links_cycling.osm.pbf'),
)

read transport network cycling


In [4]:
print('read transport network cycling after')
transport_network_cycling_after = r5py.TransportNetwork(
    os.path.join(outputs_path, 'after_oneway_links_cycling.osm.pbf'),
)

read transport network cycling after


In [5]:
print('read transport network car')
transport_network_car = r5py.TransportNetwork(
    os.path.join(outputs_path, 'tt.osm.pbf'),
)

read transport network car


In [12]:
perimeter = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), index='id'
).geometry['ebc_zrh_v01'].simplify(500, preserve_topology=True)
statent_in_perimeter = statent[statent.within(perimeter)]

In [8]:
import warnings
warnings.filterwarnings('ignore')

cells = list(statent_in_perimeter['id'].sample(n=1000, random_state=0))

def miniwrapper(cell):
    print(f'calculating cell {cell}')
    return snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_car,
        transport_network_car,
        transport_network_cycling,
        statpop, statent, cell,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.02
    )

accessibility = pd.concat(
    map(miniwrapper, cells),
    ignore_index=True
)

print('exporting file')
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, 'accessibility.gpkg')
)

calculating cell 68782482
calculating cell 68132496
calculating cell 68412666
calculating cell 68432474
calculating cell 68782456
calculating cell 67952487
calculating cell 70242463
calculating cell 69442497
calculating cell 67472490
calculating cell 67952354
exporting file


In [16]:
import warnings
warnings.filterwarnings('ignore')

cells = list(statent_in_perimeter['id'].sample(n=1000, random_state=0))

def miniwrapper(cell):
    print(f'calculating cell {cell}')
    return snman.accessibility.calculate_accessibility_for_statent_cell(
        transport_network_default,
        transport_network_car,
        transport_network_cycling_after,
        statpop, statent, cell,
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.02
    )

accessibility_after = pd.concat(
    map(miniwrapper, cells),
    ignore_index=True
)

print('exporting file')
snman.io.export_gdf(
    accessibility_after,
    os.path.join(outputs_path, 'accessibility_after.gpkg')
)

calculating cell 68782482
calculating cell 68132496
calculating cell 68412666
calculating cell 68432474
calculating cell 68782456
calculating cell 67952487
calculating cell 70242463
calculating cell 69442497
calculating cell 67472490
calculating cell 67952354
calculating cell 69242485
calculating cell 68212460
calculating cell 68222455
calculating cell 68942503
calculating cell 68692525
calculating cell 68082518
calculating cell 68672605
calculating cell 69532346
calculating cell 69702336
calculating cell 67572498
calculating cell 68432410
calculating cell 68942501
calculating cell 67912376
calculating cell 68382406
calculating cell 69712282
calculating cell 68172418
calculating cell 68352515
calculating cell 69982408
calculating cell 66832470
calculating cell 68242738
calculating cell 68932519
calculating cell 69212320
calculating cell 68632415
calculating cell 68882551
calculating cell 70232414
calculating cell 68722419
calculating cell 69442467
calculating cell 66402601
calculating 

In [12]:
import warnings
warnings.filterwarnings('ignore')

accessibility_contributions = snman.accessibility.calculate_accessibility_for_statent_cell(
    transport_network_default,
    transport_network_car,
    transport_network_cycling_after,
    statpop, statent, 69602294,
    distance_limit=50*1000,
    population_sample=0.05,
    destinations_sample=0.0166,
    sum_accessibility_contributions=False
)

snman.io.export_gdf(
    accessibility_contributions,
    os.path.join(outputs_path, 'accessibility_contributions.gpkg')
)

In [17]:
accessibility

,accessibility,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,geometry
0,4396.422581,11,2,1,-2,-2,68432474.0,POINT (2684338.001 1247404.001)
1,4017.607357,81,2,2,-2,-2,68782456.0,POINT (2687768.001 1245636.001)
2,3747.073127,5,1,1,-2,-2,68782456.0,POINT (2687781.001 1245628.001)
3,3919.844548,64,2,4,-2,-2,70242463.0,POINT (2702381.001 1246271.001)
4,3922.642433,66,1,2,-2,-2,70242463.0,POINT (2702356.001 1246313.001)
...,...,...,...,...,...,...,...,...
2760,3825.020666,57,2,2,-2,-2,67192406.0,POINT (2671854.001 1240591.001)
2761,3822.387353,36,1,2,-2,-2,67762627.0,POINT (2677577.001 1262718.001)
2762,3927.689252,59,1,2,-2,-2,67762627.0,POINT (2677584.001 1262653.001)
2763,3903.901796,53,2,2,301,3,67762627.0,POINT (2677583.001 1262713.001)


In [28]:
accessibility_before_after = pd.merge(
    accessibility, accessibility_after[['accessibility']], left_index=True, right_index=True, suffixes=['_before', '_after']
)

accessibility_before_after['accessibility_after-before'] =(
        accessibility_before_after['accessibility_after'] - accessibility_before_after['accessibility_before'])

snman.io.export_gdf(
    accessibility_before_after,
    os.path.join(outputs_path, 'accessibility_before_after.gpkg')
)